In [1]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /home/jakub/.local/lib/python3.8/site-packages (1.3.0)


In [2]:
import numpy as np
import pandas as pd
import sklearn

In [3]:
def init_weight_and_bias(X, M):
    D = X.shape[1]
    W = np.random.randn(D, M) / np.sqrt(D)
    b = np.zeros(M)
    return W.astype(np.float32), b.astype(np.float32)

In [4]:
def init_filter(shape, poolsz):
    w = np.random.randn(*shape) * np.sqrt(2) / np.sqrt(np.prod(shape[1:]) + shape[0]*np.prod(shape[2:] / np.prod(poolsz)))
    return w.astype(np.float32)

In [5]:
def relu(x):
    return x*(x>0)

In [6]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [7]:
def softmax(a):
    expA = np.exp(a)
    return expA / expA.sum(axis=1, keepdims=True)

In [8]:
def sigmoid_cost(T, Y):
    return -(T*np.log(Y) + (1-T)*np.log(1-Y)).sum()

In [9]:
def cost(T, Y):
    return -(T*np.log(Y)).sum()

In [ ]:
def cost2(T, Y):
    N = len(T)
    return -np.log(Y[np.arange(N), T]).mean()

In [10]:
def error_rate(T, Y):
    return np.mean(T != Y)

In [11]:
def y2indicator(Y):
    N = Y.shape[0]
    K = len(set(Y))
    y_indicator = np.zeros((N, K))
    for i in range(N):
        y_indicator[i, int(Y[i])] = 1
    return y_indicator

In [12]:
def get_data(balance_ones=True, Ntest=1000):
    Y = []
    X = []
    first = True
    with open('dataset/icml_face_data.csv') as file:
        for line in file.readlines():
            line = line.split(',')
            if first:
                first = False
            else:
                Y.append(int(line[0]))
                X.append([int(value) for value in line[2].split(' ')])


    Y = np.array(Y, dtype=np.float32)
    X = np.array(X, dtype=np.float32) / 255.0
    
    X, Y = sklearn.utils.shuffle(X, Y)
    
    Xtrain, Ytrain = X[:-Ntest], Y[:-Ntest]
    Xtest, Ytest = X[-Ntest:], Y[-Ntest:]
    
    if balance_ones:
        X0 = Xtrain[Ytrain != 1, :]
        Y0 = Ytrain[Ytrain != 1]
        X1 = Xtrain[Ytrain == 1, :]
        X1 = Ytrain[Ytrain == 1]
        X1 = np.repeat(X1, 9, axis=0)
        print(Y0.shape)
        print(X0.shape)
        # Xtrain = np.vstack([X0, X1])
        # Ytrain = np.concatenate((Y0, [1]*len(X1)))

    return Xtrain, Ytrain, Xtest, Ytest

In [13]:
def getImageData():
    Xtrain, Ytrain, Xtest, Ytest = getData()
    N, D = Xtrain.shape
    d = int(np.sqrt(D))
    Xtrain = Xtrain.reshape(-1, 1, d, d)
    Xvalid = Xvalid.reshape(-1, 1, d, d)
    return Xtrain, Ytrain, Xtest, Ytest

In [14]:
def get_binary_data(balance_ones=True, Ntest=1000):
    Y = []
    X = []
    first = True
    with open('dataset/icml_face_data.csv') as file:
        for line in file.readlines():
            line = line.split(',')
            if first:
                first = False
            else:
                y = int(line[0])
                if y == 0 or y == 1:
                    Y.append(int(line[0]))
                    X.append([int(value) for value in line[2].split(' ')])


    Y = np.array(Y, dtype=np.float32)
    X = np.array(X, dtype=np.float32) / 255.0
    
    X, Y = sklearn.utils.shuffle(X, Y)

    Xtrain, Ytrain = X[:-Ntest], Y[:-Ntest]
    Xtest, Ytest = X[-Ntest:], Y[-Ntest:]
    
    # if balance_ones:
    #     X0 = Xtrain[Ytrain != 1, :]
    #     Y0 = Ytrain[Ytrain != 1]
    #     X1 = Xtrain[Ytrain == 1, :]
    #     X1 = Ytrain[Ytrain == 1]
    #     X1 = np.repeat(X1, 9, axis=0)
    #     Xtrain = np.vstack([X0, X1])
    #     Ytrain = np.concatenate((Y0, [1]*X1.shape[0]))

    return Xtrain, Ytrain, Xtest, Ytest
    

In [15]:
def crossValidation(model, X, Y, K=5):
    # split data into K parts
    X, Y = shuffle(X, Y)
    sz = len(Y) // K
    errors = []
    for k in range(K):
        xtr = np.concatenate([ X[:k*sz, :], X[(k*sz + sz):, :] ])
        ytr = np.concatenate([ Y[:k*sz], Y[(k*sz + sz):] ])
        xte = X[k*sz:(k*sz + sz), :]
        yte = Y[k*sz:(k*sz + sz)]

        model.fit(xtr, ytr)
        err = model.score(xte, yte)
        errors.append(err)
    print("errors:", errors)
    return np.mean(errors)